# Welcome to the Berkeley Lab Bottom-Up (LBL-BU) Price-Responsive Home-Automation Demand-Respose Framework

Please adjust the following parameters as you wish,
or just run the examples.

## Parameters to adjust

### Customer Data / Demand Profiles

Please select a folder that includes a file cluster_summary.csv that describes the customer clusters
regarding specific criteria (sector, IOT (util), buidling_type, LCA, climate, size, ...).

Parameters: 
* Year (2025, 2030)
* Weather Year (1in2, 1in10)
* Demand Increase (midDemand, highDemand, lowDemand)

`cluster_input_folder = "/mnt/sdb/phase4_inputs/forecasted_load/non_anonymized-1in10-midDemand-with_EE_FS-2030/"`

### Otherwise Applicable Tariffs

Please provide a mapping between customer clusters (CC) and otherwise applicable tariffs (OAT) 
and the OAT files in the OpenEI json format.

This mapping is used to determine the OAT for each individual CC.

Parameters:
```
{ 
    'filter_PGE_com_B-1': {
        'filters': [
            ('util', 'equals', 'pge'), 
            ('sector', 'equals', 'com'), 
            ('ts_kw_ann_peak', '<', '75')
        ], 'tou_rate': 'B-1'},
    'filter_PGE_com_B-10': {
        'filters': [
            ('util', 'equals', 'pge'), 
            ('sector', 'equals', 'com'), 
            ('ts_kw_ann_peak', '>=', '75'),
            ('ts_kw_ann_peak', '<',  '200'),
        ], 'tou_rate': 'B-10'},
     ...
}
```

### Dynamic Rate

The dynamic rate is calculated as a function of
* the wholesale price
* system level net load
* system level gross load
* original utility revenue per price component
* original utility revenue per customer cluster
* a complex iteration scheme that estimates response to the price and then calculates a new price

Please provide the path the to dynamic rate
`wholesale_price_location = /mnt/sdb/dynamic_pricing/tariffs/PGE_2018-2019.csv`
`dynamic_rate_location = /mnt/sdb/dynamic_pricing/tariffs/PGE_2018-2019.csv`
`utility_revenue_location = /mnt/sdb/dynamic_pricing/tariffs/PGE_2018-2019.csv`
`system_level_loads_location = /mnt/sdb/dynamic_pricing/tariffs/PGE_2018-2019.csv`
`iteration_max = 1`


### Enduses and Technology Definitions

```
[
    { 
        'technology': 'HPWH',
        'enduse': 'water_heating',
        'rte': .9,
        'shift_window': 3,
        'base_load_frac': .25,
        'shift_direction': 'before',
        'storage_capacity': '100',
        'storage_loss_per_hour': '50%'
    }
]
```

### Weather and Building Envelope

```
[
    {
        'name': 'building_com',
        'filters': [
            ('util', 'equals', 'pge'), 
            ('sector', 'equals', 'com'), 
            ('enduse', 'equals', 'water_heating')
        ], 
        'storage_loss_hourly_frac': .5,
    },
    {
        'name': 'building_res',
        'filters': [
            ('util', 'equals', 'pge'), 
            ('sector', 'equals', 'res'), 
            ('enduse', 'equals', 'water_heating')
        ], 
        'storage_loss_hourly_file_8760': 
            '/mnt/sdb/dynamic_pricing/building_shell/res_thermal_loss_8760.csv',
    },
]
```

In [ ]:
# Customer Data / Demand Profiles
cluster_input_folder = "/mnt/sdb/phase4_inputs/forecasted_load/non_anonymized-1in10-midDemand-with_EE_FS-2030/"

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#############################################################################
import pandas as pd
import numpy as np
from pdb import set_trace as bp
import os

import sys
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from datetime import datetime
from datetime import timedelta 
from datetime import datetime as dt

import seaborn as sns



#sys.path.append('../cpuc-dynamic-pricing/')
#sys.path.append('../cpuc-dynamic-pricing/cpuc_dynamic_pricing/')
from Dot_Progress_Bar import Dot_Progress_Bar

In [ ]:
cluster_summary_path = cluster_input_folder+"cluster_summary.csv"
cluster_summary = pd.read_csv(cluster_summary_path)

print('{} clusters loaded.'.format(len(cluster_summary)))

In [ ]:
cl_df = dict()
i = 0
for row in cluster_summary.iterrows():
    cluster = row[1]['name']
    cl_df[cluster] = pd.read_csv(cluster_input_folder + cluster + '.csv')
    if 'water_heating' in cl_df[cluster].columns and i > 250:
        break
    i += 1
    print('.', end='')

In [ ]:
day_slice = slice(24*90,24*91)

one_day = cl_df[cluster].iloc[day_slice]

In [ ]:
one_day

In [ ]:
one_day

In [ ]:
sns.barplot(data=one_day, y='cooling', x=one_day.index)
plt.show()


In [ ]:
tech = {
    'rte': .9,
    'shift_window': 3,
    'base_load_frac': .25,
    'shift_direction': 'before',
}

RteDefinitionConstant = {
    'type': 'fractional-constant',
    'value': .9
}

import random
random.seed(876000001234)
RteDefinition8760 = {
    'type': 'fractional-8760',
    'value': [random.randint(50, 90) / 100 for x in range(0, 8760)]
    # This simulates the weather
}


#restrictions
#   'max_load': per enduse the maximum demand
#        
#        Empirical values per enduse / technology

In [ ]:
price = pd.read_csv('/mnt/sdb/dynamic_pricing/tariffs/PGE_2018-2019.csv')[-8760:].reset_index(drop=True)
display(price.head())
display(price[day_slice].head())

In [ ]:
def select_cluster_and_day(cl_df):
    for cluster, df in cl_df.items():
        print('.', end='')
        if 'water_heating' not in df.columns:
            continue
        for day in range(0, 365):
            day_slice = slice(day*24, (day + 1)*24)
            if df.loc[day_slice, 'heating'].min() == 0:
                continue
            if df.loc[day_slice, 'cooling'].max() < 0.01:
                continue
            
            return cluster, day_slice
        

cluster, day_slice = select_cluster_and_day(cl_df)
print(cluster, day_slice)

one_day = cl_df[cluster][day_slice]

# TODO: Select a cluster from each loadshape (clustered by Sam)



In [ ]:
def plot_load_vs_price(cluster, day_slice, enduse):
    ts = cl_df[cluster][day_slice]
    if enduse not in ts.columns:
        return
        
    plt.figure(figsize=(5,2))
    ax = sns.barplot(data=ts, y=enduse, x=one_day.index)

    ax.set_ylabel(f'{enduse.capitalize()} Load (kW)')
    ax2 = ax.twinx()
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax2.xaxis.set_major_locator(plt.MultipleLocator(6))

    plt.gcf().patch.set_facecolor('white')

    sns.lineplot(data=price[day_slice], x=range(0,24), y='Total', ax=ax2, label='Dynamic Rate', color='black', linestyle='dashed')
    ax2.set_ylabel('$ / kWh')

    plt.show()


print(cluster)
for enduse in ['cooling', 'heating', 'water_heating']:
     plot_load_vs_price(cluster, day_slice, enduse)
            
            
            

In [ ]:
plt.figure(figsize=(6,2))
ax = sns.barplot(data=one_day, y='heating', x=one_day.index)

ax.set_ylabel('Heating Load (kW)')
ax2 = ax.twinx()
ax.xaxis.set_major_locator(plt.NullLocator())
ax2.xaxis.set_major_locator(plt.MultipleLocator(6))

sns.lineplot(data=price[day_slice], x=range(0,24), y='Total', ax=ax2, label='Dynamic Rate')
ax2.set_ylabel('$ / kWh')

plt.gcf().patch.set_facecolor('white')

plt.show()


In [ ]:
print(one_day.index)
one_day
print(price[day_slice].index)

In [ ]:
print(price)

In [ ]:
def plot_current_action(one_day, shift_schedule, enduse, take_hour, shed_hour, price, balance_take_shed = False):
    plt.figure(figsize=(5,2))
    before_after = shift_schedule.copy()[['take', 'final', 'shed', 'hour']]
    before_after['shed'] = -1 * before_after['shed']
    
    if balance_take_shed:
        c_shed_larger_take = before_after['shed'] > before_after['take']
        before_after.loc[c_shed_larger_take, 'shed'] -= before_after.loc[c_shed_larger_take, 'take']
        before_after.loc[c_shed_larger_take, 'take'] = 0

        c_take_larger_shed = before_after['shed'] < before_after['take']
        before_after.loc[c_take_larger_shed, 'take'] -= before_after.loc[c_take_larger_shed, 'shed']
        before_after.loc[c_take_larger_shed, 'shed'] = 0

    #display(before_after)
    if balance_take_shed:
        before_after['final'] -= before_after['take']
        ax = before_after[['final',  'take', 'shed', 'hour']].plot.bar(stacked=True, x='hour', color=['#1f77b4', '#ff7f0e', '#2ca02c'], ax=plt.gca())
        before_after['final2'] = before_after['final'] + before_after['take']
        before_after['orig2'] = before_after['final'] + before_after['shed']
        #ax.fill_between(range(0,24), before_after['final2'], 0, where=0 < before_after['final2'], facecolor='gold', interpolate=True)
        #ax.fill_between(range(0,24), before_after['orig2'], 0, where=0 < before_after['orig2'], facecolor='#33ffaa', interpolate=True)
        ax.fill_between(range(0,24), before_after['final'], 0, where=0 < before_after['final'], facecolor='#3fa7d4', interpolate=True)
        sns.lineplot(data=before_after, x=range(0,24), y='final2', color='gold')
        sns.lineplot(data=before_after, x=range(0,24), y='orig2', color='#3fa7d4', linewidth=3)
        sns.lineplot(data=before_after, x=range(0,24), y='final', color='#33ffaa')
        #'#1f77b4', '#ff7f0e'
    else:
        before_after['final'] -= before_after['shed']
        before_after['shed'] = -1 * before_after['shed']
        #ax = before_after[['final',  'take', 'shed', 'hour']].plot.bar(x='hour', )
        melted = before_after.melt('hour', var_name='type', value_name='value')
        #display(melted)
        ax = sns.barplot(data=melted, y='value', x='hour', hue='type')

    ax.set_ylabel(f'{enduse.capitalize()} Load (kW)')
    plt.gca().legend(loc='lower right')
    ax2 = ax.twinx()
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax2.xaxis.set_major_locator(plt.MultipleLocator(6))
    #ax.legend(loc='upper left', bbox_anchor=(1.01,1))
    #ax.legend(loc='upper left')
    
    sns.lineplot(data=price[day_slice], x=range(0,24), y='Total', ax=ax2, label='Dynamic Rate', color='black', linestyle='dashed')
    ax2.set_ylabel('$ / kWh')

    plt.gcf().patch.set_facecolor('white')
    plt.gca().legend(loc='lower left')

    plt.show()
    

In [ ]:
def cost_analysis(one_day, shift_schedule, enduse, price):
    before_after = shift_schedule.copy()[['orig', 'take', 'final', 'shed', 'hour']]
    before_after['shed'] = -1 * before_after['shed']
    before_after['Cost_Orig'] = before_after['orig'] * price.loc[one_day.index].reset_index()['Total']
    before_after['Cost_Final'] = before_after['final'] * price.loc[one_day.index].reset_index()['Total']
    before_after['Cost_Diff'] = before_after['Cost_Final'].cumsum() - before_after['Cost_Orig'].cumsum()
    before_after['Load_Diff'] = before_after['final'].cumsum() - before_after['orig'].cumsum()
    return before_after

In [ ]:
def plot_cost(one_day, shift_schedule, enduse, price):
    before_after = cost_analysis(one_day, shift_schedule, enduse, price)
    
    before_after = shift_schedule.copy()[['orig', 'take', 'final', 'shed', 'hour']]
    before_after['shed'] = -1 * before_after['shed']
    before_after['Cost_Orig'] = before_after['orig'] * price.loc[one_day.index].reset_index()['Total']
    before_after['Cost_Final'] = before_after['final'] * price.loc[one_day.index].reset_index()['Total']
    before_after['Cost_Diff'] = before_after['Cost_Final'].cumsum() - before_after['Cost_Orig'].cumsum()
    before_after['Load_Diff'] = before_after['final'].cumsum() - before_after['orig'].cumsum()
    
    plt.figure(figsize=(5,2))
    plt.gcf().patch.set_facecolor('white')
    
    sns.lineplot(data=before_after, x=range(0,24), y='Cost_Final', color='gold', label='Final Costs')
    sns.lineplot(data=before_after, x=range(0,24), y='Cost_Orig', color='green', label='Original Costs')
    sns.lineplot(data=before_after, x=range(0,24), y='Cost_Diff', color='darkblue', label='Bill Diff')
    sns.lineplot(data=before_after, x=range(0,24), y='Load_Diff', color='orange', label='Load Diff', linestyle='dashed')
    
    plt.gca().legend(loc='lower left')
    
    plt.show()
    
    display(before_after)
    
    return cost_analysis
    

In [ ]:
from Tech_Models.basicshifter import Basic_Shifter

# optimizing_strategy1 is reverse loop
#print(day_slice)
#print(day_slice.stop)

take_hour = 0
shed_hour = 0

enduse = 'cooling'
for enduse in ['cooling', 'heating', 'water_heating']:
    shift_schedule = Basic_Shifter.get_shift_schedule(one_day, day_slice, enduse, tech, price)
    
    schedule_to_plot = shift_schedule.melt('hour', var_name='type', value_name='value')
    #display(schedule_to_plot[schedule_to_plot['hour'] == 929])

    plot_load_vs_price(cluster, day_slice, enduse)
    plot_current_action(one_day, shift_schedule, enduse, take_hour, shed_hour, price)
    plot_current_action(one_day, shift_schedule, enduse, take_hour, shed_hour, price, balance_take_shed=True)
    plot_cost(one_day, shift_schedule, enduse, price)
    break

In [ ]:
schedule_to_plot = shift_schedule.melt('hour', var_name='type', value_name='value')
#schedule_to_plot.index = schedule_to_plot.hour

In [ ]:
schedule_to_plot

In [ ]:
enduse = 'EndUseName'
day_slice = slice(0,24)

demand = [8 for x in range(24)]
demand_df = pd.DataFrame.from_dict({enduse: demand})

price = [5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 8, 8, 8, 8]
price_df = pd.DataFrame.from_dict({'Total': price})

tech = {
    'rte': .9,
    'shift_window': 3,
    'base_load_frac': .25,
    'shift_direction': 'before',
}

shift_schedule = Basic_Shifter.get_shift_schedule(demand_df, day_slice, enduse, tech, price_df)

take_hour = 0
shed_hour = 0

#plot_load_vs_price(cluster, day_slice, enduse)
plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df)
plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df, balance_take_shed=True)
plot_cost(demand_df, shift_schedule, enduse, price_df)

In [ ]:
shift_schedule.loc[14, 'final']

In [ ]:
enduse = 'EndUseName'
day_slice = slice(0,24)

demand = [8 for x in range(24)]
demand_df = pd.DataFrame.from_dict({enduse: demand})

price = [5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 8, 8, 8, 8]
price_df = pd.DataFrame.from_dict({'Total': price})

tech = {
    'rte': .2,
    'shift_window': 3,
    'base_load_frac': .25,
    'shift_direction': 'before',
}

shift_schedule = Basic_Shifter.get_shift_schedule(demand_df, day_slice, enduse, tech, price_df)

take_hour = 0
shed_hour = 0

#plot_load_vs_price(cluster, day_slice, enduse)
plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df)
plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df, balance_take_shed=True)
plot_cost(demand_df, shift_schedule, enduse, price_df)

shift_schedule.iloc[-1]


In [ ]:
enduse = 'EndUse'
day_slice = slice(0,24)

demand = [24]
demand.extend([8 for x in range(23)])
demand_df = pd.DataFrame.from_dict({enduse: demand})

price = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 5, 10, 9, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
price_df = pd.DataFrame.from_dict({'Total': price})

tech = {
    'rte': .5,
    'shift_window': 3,
    'base_load_frac': 1,
    'shift_direction': 'before',
}
shift_schedule = Basic_Shifter.get_shift_schedule(demand_df, day_slice, enduse, tech, price_df)
print(shift_schedule.loc[13, 'final']) 
print(shift_schedule.loc[14, 'final']) 
print(shift_schedule.loc[15, 'final']) 
print(shift_schedule.loc[16, 'final'])

plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df)
plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df, balance_take_shed=True)
plot_cost(demand_df, shift_schedule, enduse, price_df)

In [ ]:
enduse = 'EndUseName'
day_slice = slice(0,24)

demand = [8 for x in range(24)]
demand_df = pd.DataFrame.from_dict({enduse: demand})

price = [i for i in range(24)]
price_df = pd.DataFrame.from_dict({'Total': price})

tech = {
    'rte': .1,
    'shift_window': 3,
    'base_load_frac': 1,
    'shift_direction': 'before',
}
shift_schedule = Basic_Shifter.get_shift_schedule(demand_df, day_slice, enduse, tech, price_df)
print(shift_schedule.loc[13, 'final']) 
print(shift_schedule.loc[14, 'final']) 
print(shift_schedule.loc[15, 'final']) 
print(shift_schedule.loc[16, 'final'])

plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df)
plot_current_action(demand_df, shift_schedule, enduse, take_hour, shed_hour, price_df, balance_take_shed=True)
plot_cost(demand_df, shift_schedule, enduse, price_df)